In [1]:
import shutil
import os
import netCDF4 as nc
import pandas as pd

#### Stages to make Latin Hypercube

 - Make an ensemble of files
 - Copy one of the SP calibration outputs into each of these file
 - Make a list of variables to modify
 - Create actual hypercube
 - Define the ranges of the parameters using the csv file
 - Apply to parameter list 

#### Control paths

In [6]:
# Averaged SP calibrated values, from paraemter_averaging and paraemeter_merge scripts. 
averaged_SP_params_files = '../fates_params_vertsc_dayl_SPcal.nc' 

# Where to find the list of parameters and their modifications
parameter_list_file = '../NOCOMP_PPE_parameters.csv'
pft_parameter_list = '../pft_specific_parameter_inputs/PFT_params.csv'

# Adjusted default x calibration file to make a reasonable baseline. 
modified_baseline_param_files='../modified_average_file.nc'

# What is the path to the new parameter file ensemble 
pdirroot='../paramfiles/nocomp_oaat_v5_'

#### Read the main parameter list CSV file into a pandas DataFrame

In [8]:
df = pd.read_csv(parameter_list_file)
fates_parameter_list = df['fates_parameter'].tolist()
minp = df['min'].tolist() 
maxp = df['max'].tolist() 
dimp = df['dim'].tolist()

#### Read the PFT parameter list CSV file into a pandas DataFrame

In [10]:
dfp = pd.read_csv(pft_parameter_list)
pftindex = dfp['pft_index'].tolist()
pftparam = dfp['param'].tolist() 
pftmin = dfp['min'].tolist()
pftmax = dfp['max'].tolist()

#### map from the variables in the pftvariable csv file into the fates pft space

In [12]:
string_mapping = {
    "fates_rad_stem_rhovis": "rhosvis",
    "fates_rad_stem_rhonir": "rhosnir",
    "fates_rad_leaf_rhonir": "rholnir",
    "fates_rad_leaf_rhovis": "rholvis",
    "fates_rad_leaf_taunir": "taulnir",
    "fates_rad_leaf_tauvis": "taulvis",
    "fates_leaf_stomatal_slope_medlyn": "medlynslope",
    "fates_leaf_slatop": "sla",
    "fates_rad_leaf_xl": "xl"   
}

#### Retreive the full list of the parameters 

In [14]:
reference_vars = nc.Dataset(modified_baseline_param_files, 'r+') 
fatespfts=reference_vars.dimensions['fates_pft']
print(fatespfts.size)


12


#### List of parameters to perturb 
###### Going to do this manually for now, just because. 

In [38]:
nocomp_lhc_params = {'fates_leaf_slatop','fates_allom_d2bl1','fates_allom_d2cm_coefficient_max'
                    }

#### Make a latin hypercube

In [40]:
import numpy as np

# Define the number of samples and variables
num_samples = 20
num_variables = 3

# Generate a Latin Hypercube
hypercube = np.zeros((num_samples, num_variables))

for i in range(num_variables):
    hypercube[:, i] = np.random.permutation(np.linspace(0, 1, num_samples))

#print(hypercube)

#### Modify variables in LHC ensemble according to the values in the input csv fil

In [19]:
for i in range(1,nlhc_samples)
    counter = 0
    newfile = pdirroot + str(i) + '.nc' 
    shutil.copy(modified_baseline_param_files, newfile)
    nc_file = nc.Dataset(newfile, 'r+')
    
    for index, parameter in enumerate(nocomp_lhc_params):
        if parameter in reference_vars.variables:
            # Perform operations on each parameter
            print('#',index, parameter)  # Replace with your operations
            variable_to_modify = nc_file.variables[parameter]
                
            if minp[index] == '50percent':
                if variable_to_modify.dtype == int:
                    variable_data = variable_to_modify[:]
                    variable_data = (variable_data * 0.5).astype(int)
                    variable_to_modify[:] = variable_data  
                else: #not integer 
                    min_range = variable_data * 0.5    
            elif minp[index] == '100percent':
                    min_range[:] = variable_data * 0            
            elif minp[index] ==  'pft':
                shortvar = string_mapping[parameter]
                table = dfp[dfp['param'] == shortvar]
                pftmins=table['min'].tolist()
                variable_to_modify[:] *=  0
               
                for p in range(1, 12 ):               
                    variable_to_modify[p-1] += float(pftmins[p-1])
                #variable_to_modify[12] += float(pftmins[10])
                #variable_to_modify[13] += float(pftmins[11])
                print(shortvar, ' table ',pftmins)
                print(shortvar, counter ,'min ',variable_to_modify[:])
            else:
                print('minp',minp[index])
                variable_to_modify[:] *=  0
                variable_to_modify[:] += float(minp[index])
            
            nc_file.close()  # Close the modified file
            
        else: #doenst exist 
            print('variable is not in file', parameter)

# 0 fates_alloc_storage_cushion
pft min [0.6 1.2 1.2 0.6 1.2 1.2 1.2 1.2 1.2 1.2 1.2 1.2]
pftmax [1.8 3.6 3.6 1.8 3.6 3.6 3.6 3.6 3.6 3.6 3.6 3.6]
# 1 fates_allom_agb_frac
pft min [0.3 0.3 0.3 0.3 0.3 0.3 0.3 0.3 0.3 0.3 0.3 0.3]
pftmax [0.9 0.9 0.9 0.9 0.9 0.9 0.9 0.9 0.9 0.9 0.9 0.9]
# 2 fates_allom_agb1
pft min [0.03365    0.0682006  0.01965285 0.13268475 0.03365    0.0364349
 0.03448    0.03448    0.03448    0.005      0.005      0.005     ]
pftmax [0.10095    0.2046018  0.05895855 0.39805425 0.10095    0.1093047
 0.10344    0.10344    0.10344    0.015      0.015      0.015     ]
# 3 fates_allom_agb2
pft min [0.488      0.47245205 0.5436675  0.41606605 0.488      0.51866055
 0.286      0.286      0.286      0.286      0.286      0.286     ]
pftmax [1.464      1.41735615 1.6310025  1.24819815 1.464      1.55598165
 0.858      0.858      0.858      0.858      0.858      0.858     ]
# 4 fates_allom_agb3
pft min [0.97 0.97 0.97 0.97 0.97 0.97 0.97 0.97 0.97 0.97 0.97 0.97]
pftmax [2.91